In [40]:
import torch
from kan import *
from my_functions import calculate_signature, calculate_signature_over_all_surface, format_signature
import plotly.graph_objects as go


In [41]:
mesh_path = "data/bunny_curvs.ply"
sampled_indices = random.sample(range(0, 106712), 10)
print(sampled_indices)


[50494, 99346, 55125, 5306, 33936, 67013, 63691, 53075, 102734, 39755]


In [42]:

training_num_per_sampled_surface = 10
signatures = torch.zeros((len(sampled_indices), training_num_per_sampled_surface, 6))
means = []
stds = []

grid_range = 1.0
ball_radius = 0.07
lr = 0.1
for i,sampled_index in enumerate(sampled_indices):
    print(i)
    dataset = create_dataset_from_mesh(mesh_path=mesh_path,sampled_index=sampled_index, show_sample=False, ball_radius=ball_radius, seed=sampled_index, down_sampling_ratio=0.8)
    print(dataset['train_input'].shape, dataset['train_label'].shape)
    for j in range(training_num_per_sampled_surface):
        model = KAN(width=[2,5,1], grid=3, k=5, grid_eps=1.0, seed=0, grid_range=[-grid_range,grid_range], noise_scale=0.1, noise_scale_base=1.0, base_fun=torch.nn.SiLU(), learn_rotation_mat=False)
        model.train(dataset, opt="LBFGS", steps=20,lr=lr, lamb=100.0, lamb_l1=1.0, lamb_entropy=1.0, lamb_coef=0.0, lamb_coefdiff=0.0, sglr_avoid=True)
        model = model.prune(threshold=3e-3, mode='manual', active_neurons_id=[[0,1],[0]])
        model.train(dataset, opt="LBFGS", steps=50,lr=lr, sglr_avoid=True)
         # manual mode
        # fix all
        model.fix_symbolic(0,0,0,'x^4')
        model.fix_symbolic(0,1,0,'x^3')
        model.fix_symbolic(1,0,0,'x^2')

        model.train(dataset, opt="LBFGS", steps=50, lr=lr,sglr_avoid=True, lamb_coef=1.0, lamb_coefdiff=1.0, loss_fn=gaussian_weighted_mse)

        print(model.symbolic_formula()[0][0])
        x = torch.tensor([[0.0], [0.0]], requires_grad=True).T

        signature = calculate_signature(model, x) # only calc sig in the origin
        print("sampled_index", i, "exp ", j, "signature ", signature)
        signatures[i,j,:] = signature



0
Number of vertices: 106712
torch.Size([131, 2]) torch.Size([131, 1])


train loss: 1.62e-02 | test loss: 1.76e-02 | reg: 2.68e+00 : 100%|██| 20/20 [00:02<00:00,  9.75it/s]
train loss: 1.19e-03 | test loss: 1.22e-02 | reg: 6.58e+00 : 100%|██| 50/50 [00:02<00:00, 18.21it/s]


r2 is 0.9938840866088867
r2 is 0.9844156503677368
Best value at boundary.
r2 is 0.7579801678657532
r2 is not very high, please double check if you are choosing the correct symbolic function.


train loss: 1.72e-03 | test loss: 1.05e-02 | reg: 8.79e+00 : 100%|██| 50/50 [00:01<00:00, 48.50it/s]
/Users/gal.yona/Documents/MSc/Thesis/ImplicitNeuralSurfaces/forked_pykan/pykan/kan/KAN.py:344: UserWarning:

std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/ReduceOps.cpp:1760.)



-22751.17*(0.72*(0.06 - x_2)**3 + (-x_1 - 0.17)**4)**2
sampled_index 0 exp  0 signature  tensor([[   10.0367,    13.0071,   -18.7962,  -549.8358,   161.9982, -5560.9087]],
       grad_fn=<StackBackward0>)


...

1.83 - 1135.74*(-(-x_1 - 0.03)**4 + 0.03*(-x_2 - 0.15)**3 - 0.04)**2
sampled_index 9 exp  8 signature  tensor([[   1.8370,    2.9568,  -35.7400,    6.5525, -177.5191,   14.7787]],
       grad_fn=<StackBackward0>)


train loss: 3.91e-02 | test loss: 4.52e-02 | reg: 2.58e+00 : 100%|██| 20/20 [00:01<00:00, 10.35it/s]
train loss: 1.46e-03 | test loss: 3.76e-03 | reg: 3.07e+00 : 100%|██| 50/50 [00:02<00:00, 17.65it/s]


r2 is 0.9916370511054993
r2 is 0.9989679455757141
r2 is 0.8535221219062805
r2 is not very high, please double check if you are choosing the correct symbolic function.


train loss: 2.58e-03 | test loss: 3.18e-03 | reg: 1.15e+01 : 100%|██| 50/50 [00:01<00:00, 41.25it/s]
/Users/gal.yona/Documents/MSc/Thesis/ImplicitNeuralSurfaces/forked_pykan/pykan/kan/KAN.py:344: UserWarning:

std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/ReduceOps.cpp:1760.)



0.41 - 59373.89*(-(-x_1 - 0.e-2)**4 + 0.01*(-x_2 - 0.13)**3)**2
sampled_index 9 exp  9 signature  tensor([[   1.5704,    0.9134,  -34.5930,    9.3697, -194.8654,    5.8357]],
       grad_fn=<StackBackward0>)


In [43]:
signatures

tensor([[[ 1.0037e+01,  1.3007e+01, -1.8796e+01, -5.4984e+02,  1.6200e+02,
          -5.5609e+03],
         [ 3.8382e+00,  1.4614e+01,  2.0539e+01, -4.4854e+01,  1.7621e+02,
          -3.3437e+02],
         [ 1.0021e+01,  1.4191e+01, -1.0828e+01, -5.2068e+02,  1.8097e+02,
          -5.4963e+03],
         [ 1.0024e+01,  1.2300e+01, -2.4200e+01, -5.6796e+02,  1.5074e+02,
          -5.5828e+03],
         [ 1.0431e+01,  1.3576e+01, -2.3924e+01, -5.5190e+02,  1.5017e+02,
          -5.7430e+03],
         [ 5.3322e+00,  2.7358e+01,  9.0555e+01, -6.0985e+01,  1.1465e+03,
          -5.0014e+02],
         [ 9.9956e+00,  1.1516e+01, -2.8937e+01, -5.8527e+02,  1.4024e+02,
          -5.5977e+03],
         [ 3.9680e+00,  1.5479e+01,  2.0423e+01, -5.2521e+01,  1.9046e+02,
          -3.9689e+02],
         [ 3.8454e+00,  1.4666e+01,  1.9731e+01, -4.6308e+01,  1.6972e+02,
          -3.4674e+02],
         [ 3.5477e+00,  1.2558e+01, -1.1199e+00, -4.1664e+01, -1.2418e+01,
          -2.9261e+02]],

        

In [44]:
#  #discard all nan and 0
# signatures = signatures[~torch.isnan(signatures).any(dim=2)]
# signatures = signatures[~(signatures == 0).all(dim=2)]

In [45]:
mean_signature = torch.mean(signatures, dim=1)
std_signature = torch.std(signatures, dim=1)
print("mean signature ", mean_signature)
print("std signature ", std_signature)

mean signature  tensor([[ 7.1040e+00,  1.4927e+01,  4.3443e+00, -3.0220e+02,  2.4546e+02,
         -2.9852e+03],
        [-5.4495e-01, -1.6232e+00, -1.1686e+02,  7.4998e+01, -1.4546e+02,
         -3.0102e+02],
        [ 8.1055e-01,  1.3781e-03, -1.3602e+00, -3.8046e+01, -5.1101e+00,
         -5.9996e-03],
        [ 4.0716e+00, -5.9031e+02, -6.2963e+01,  7.7663e+02, -4.1694e+03,
         -3.6927e+04],
        [ 2.9598e-01,  1.0352e-02, -2.0562e+00, -1.0846e+01, -1.9609e-01,
         -3.1332e+00],
        [-3.1830e+00,  7.7205e-01, -2.7006e+02,  6.3303e+01, -5.0801e+02,
         -1.3031e+02],
        [ 5.8253e+00,  5.0722e+00,  2.6707e+00,  3.1602e+02,  1.6199e+02,
          7.0631e+02],
        [ 9.0133e-01,  3.1477e-01, -4.1563e+01, -3.2191e+01, -1.3706e+02,
         -1.2371e+01],
        [ 9.0745e-01,  5.9905e-01, -1.3023e+00, -1.0127e+01, -3.6942e+00,
         -9.4130e+00],
        [ 1.8253e+00,  1.5698e+00, -3.5892e+01,  1.1916e+01, -2.2366e+02,
          1.0486e+01]], grad_fn=<Mean

In [46]:
mean_of_means_signature = torch.mean(mean_signature, dim=0)
mean_of_stds_signature = torch.mean(std_signature, dim=0)
print("mean of means signature ", mean_of_means_signature)
print("mean of stds signature ", mean_of_stds_signature)

mean of means signature  tensor([ 1.8013e+00, -5.6867e+01, -5.2504e+01,  8.4945e+01, -4.7851e+02,
        -3.9652e+03], grad_fn=<MeanBackward1>)
mean of stds signature  tensor([1.3062e+00, 3.8698e+01, 3.9960e+01, 1.2102e+02, 1.0924e+03, 3.7937e+03],
       grad_fn=<MeanBackward1>)


In [47]:
# calculate std/mean to see the variation of the signature
std_over_mean = abs(mean_of_stds_signature/mean_of_means_signature)
print("std/mean ", std_over_mean)

std/mean  tensor([0.7251, 0.6805, 0.7611, 1.4246, 2.2828, 0.9568],
       grad_fn=<AbsBackward0>)


In [49]:
outliers = torch.abs(signatures - mean_signature) > 2 * std_signature
signatures_without_outliers = signatures[~outliers.any(dim=2)]
mean_signature = torch.mean(signatures_without_outliers, dim=1)
std_signature = torch.std(signatures_without_outliers, dim=1)
print("mean signature ", mean_signature)
print("std signature ", std_signature)
print(signatures_without_outliers.shape)
print(signatures_without_outliers)




mean signature  tensor([-9.9075e+02, -9.3022e+01, -7.9698e+00, -6.8315e+00, -1.8500e+04,
        -4.3592e+00, -4.2534e+00, -3.7535e+02, -3.6346e+02,  1.8680e+02,
         1.9596e+02, -5.8530e+01, -3.1480e+01, -3.9920e+00, -3.6045e+00,
        -4.4932e+01, -3.5295e+01], grad_fn=<MeanBackward1>)
std signature  tensor([2.2523e+03, 1.4806e+02, 1.5610e+01, 1.7931e+01, 3.6729e+04, 1.0019e+01,
        1.0045e+01, 7.1817e+02, 6.5780e+02, 2.0605e+02, 2.2518e+02, 8.7735e+01,
        4.2920e+01, 5.0191e+00, 4.5714e+00, 1.0948e+02, 7.9775e+01],
       grad_fn=<StdBackward0>)
torch.Size([17, 6])
tensor([[ 1.0037e+01,  1.3007e+01, -1.8796e+01, -5.4984e+02,  1.6200e+02,
         -5.5609e+03],
        [-7.4338e-01, -1.1673e+00, -1.4446e+02,  7.9734e+01, -1.6355e+02,
         -3.2795e+02],
        [ 8.9148e-01,  1.3223e-03, -2.0604e+00, -3.9249e+01, -7.3442e+00,
         -5.8293e-02],
        [ 4.5964e-01,  1.2145e-03,  6.2954e-01, -4.3423e+01,  1.1406e+00,
          2.0332e-01],
        [ 7.1186e+00, 

In [50]:
# calculate std/mean to see the variation of the signature
std_over_mean = abs(std_signature/mean_signature)
print("std/mean ", std_over_mean)

std/mean  tensor([2.2733, 1.5917, 1.9586, 2.6247, 1.9853, 2.2984, 2.3617, 1.9133, 1.8098,
        1.1031, 1.1491, 1.4990, 1.3634, 1.2573, 1.2682, 2.4366, 2.2602],
       grad_fn=<AbsBackward0>)
